In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Maranhão - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_MA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado,Maranhão - Consumo de Cimento (t)
0,2003,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.213662,0.635570,293.705283,4.315322e+07,2.636001e+06,6.034057,3.872495e+07,470.726000
1,2004,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.145546,0.640694,308.130986,4.381448e+07,2.646202e+06,6.058375,3.895418e+07,531.005000
2,2005,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.122342,0.645312,308.093212,4.443858e+07,2.668922e+06,6.082069,3.916195e+07,579.458000
3,2006,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.038324,0.649437,322.991255,4.501666e+07,2.698856e+06,6.102754,3.934191e+07,693.939000
4,2007,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.007661,0.653077,339.260912,4.550078e+07,2.671721e+06,6.110757,3.943349e+07,816.847000
5,2008,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,7.978325,0.656244,352.164879,4.598085e+07,2.691136e+06,6.129297,3.956333e+07,963.205000
6,2009,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.007891,0.658950,363.707506,4.630053e+07,2.690232e+06,6.126304,3.956853e+07,1123.586000
7,2010,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,7.971707,0.661204,424.052860,4.751040e+07,2.700684e+06,6.153755,3.988864e+07,1378.741000
8,2011,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,7.961524,0.663017,558.996171,4.866181e+07,2.701549e+06,6.182820,4.018069e+07,1282.016000
9,2012,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,10.302202,0.664173,688.169638,4.972690e+07,2.696497e+06,6.208137,4.043865e+07,1528.770000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,8.213662,0.635570,293.705283,4.315322e+07,2.636001e+06,6.034057,3.872495e+07
1,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,8.145546,0.640694,308.130986,4.381448e+07,2.646202e+06,6.058375,3.895418e+07
2,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,8.122342,0.645312,308.093212,4.443858e+07,2.668922e+06,6.082069,3.916195e+07
3,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,8.038324,0.649437,322.991255,4.501666e+07,2.698856e+06,6.102754,3.934191e+07
4,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,8.007661,0.653077,339.260912,4.550078e+07,2.671721e+06,6.110757,3.943349e+07
5,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,7.978325,0.656244,352.164879,4.598085e+07,2.691136e+06,6.129297,3.956333e+07
6,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,8.007891,0.658950,363.707506,4.630053e+07,2.690232e+06,6.126304,3.956853e+07
7,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,7.971707,0.661204,424.052860,4.751040e+07,2.700684e+06,6.153755,3.988864e+07
8,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,7.961524,0.663017,558.996171,4.866181e+07,2.701549e+06,6.182820,4.018069e+07
9,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,10.302202,0.664173,688.169638,4.972690e+07,2.696497e+06,6.208137,4.043865e+07


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      531.005000
1      579.458000
2      693.939000
3      816.847000
4      963.205000
5     1123.586000
6     1378.741000
7     1282.016000
8     1528.770000
9     1563.125000
10    1303.411571
11    1191.509657
12    1100.217771
13     971.921000
14     750.012000
15     774.045000
16    1038.249000
17    1059.129000
18    1083.999000
19            NaN
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.828876,-2.014052,-1.096791,-1.621261,-0.224145,-1.998287,-1.870736
1,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.862473,-1.605840,-0.997896,-1.414909,-0.023082,-1.612382,-1.530907
2,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.873918,-1.237891,-0.998155,-1.220153,0.424735,-1.236379,-1.222893
3,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.915359,-0.909344,-0.896021,-1.039757,1.014736,-0.908125,-0.956105
4,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.930483,-0.619336,-0.784484,-0.888683,0.479902,-0.781118,-0.820337
5,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.944952,-0.367005,-0.696021,-0.738873,0.862576,-0.486904,-0.627858
6,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.930370,-0.151490,-0.616890,-0.639114,0.844749,-0.534398,-0.620145
7,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.948217,0.028073,-0.203191,-0.261563,1.050767,-0.098789,-0.145584
8,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.953239,0.172546,0.721915,0.097744,1.067819,0.362454,0.287365
9,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,0.201265,0.264666,1.607466,0.430115,0.968230,0.764210,0.669794


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      531.005000
1      579.458000
2      693.939000
3      816.847000
4      963.205000
5     1123.586000
6     1378.741000
7     1282.016000
8     1528.770000
9     1563.125000
10    1303.411571
11    1191.509657
12    1100.217771
13     971.921000
14     750.012000
15     774.045000
16    1038.249000
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
17,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,0.997695,1.039369,-0.622646,0.843673,-0.833746,0.580394,0.644008


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1059.129
Name: Maranhão - Consumo de Cimento (t), dtype: float64

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1355178669, 1694785108, 1049501944, 2256916787, 308269832, 3877489773, 3342014361, 1992076552, 1162751687, 4108777325, 4104875658, 527902512, 770138140, 526246180, 2404551474, 767588292, 130520392, 2959285369, 186964077, 472811970, 3624810600, 201307772, 415934642, 807449559, 3382766140, 4252707503, 1480214281, 3489389478, 3418660483, 4013190059, 1866380167, 337411155, 1835629017, 2356417838, 2762357234, 3718007907, 65931756, 2967006318, 2313036433, 3125659114, 609152261, 2082936039, 680683045, 1164185824, 4218093205, 1679944046, 2439095754, 557301380, 2787849232, 2217805656, 2190806497, 3963715053, 4230718229, 3067760955, 2892688271, 4246046797, 777753636, 609606550, 1647083026, 499930630, 2960810077, 1712187845, 2449825685, 646401963, 3502571513, 2026933007, 4241550380, 2578906686, 659013197, 3521975888, 3374868127, 3114800763, 3474738568, 2279094116, 1024213478, 3154942505, 2124144445, 1575730394, 1124383054, 51223867, 1403572664, 1068135978, 3451680982, 2767599897, 3111948874, 352

2023-10-01 13:20:23.728096: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:20:23.759137: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 11113.5673828125
winner_seed: 3877489773


Step: 6 ___________________________________________
val_loss: 17602.416015625


Step: 7 ___________________________________________
val_loss: 20851.6796875


Step: 8 ___________________________________________
val_loss: 46562.484375


Step: 9 ___________________________________________
val_loss: 57014.38671875


Step: 10 ___________________________________________
val_loss: 14127.2578125


Step: 11 ___________________________________________


2023-10-01 13:22:05.143203: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 21322.794921875


Step: 12 ___________________________________________
val_loss: 17051.67578125


Step: 13 ___________________________________________
val_loss: 19835.46875


Step: 14 ___________________________________________
val_loss: 17943.90625


Step: 15 ___________________________________________
val_loss: 14718.1298828125


Step: 16 ___________________________________________
val_loss: 41082.65625


Step: 17 ___________________________________________
val_loss: 21669.365234375


Step: 18 ___________________________________________
val_loss: 38003.86328125


Step: 19 ___________________________________________
val_loss: 18907.111328125


Step: 20 ___________________________________________
val_loss: 19566.728515625


Step: 21 ___________________________________________
val_loss: 18074.619140625


Step: 22 ___________________________________________
val_loss: 22834.3671875


Step: 23 ___________________________________________
val_loss: 22650.90625


Step: 24 __________

...

val_loss: 17051.72265625


Step: 66 ___________________________________________
val_loss: 21881.400390625


Step: 67 ___________________________________________
val_loss: 19325.453125


Step: 68 ___________________________________________
val_loss: 17070.6953125


Step: 69 ___________________________________________
val_loss: 17079.072265625


Step: 70 ___________________________________________
val_loss: 17569.376953125


Step: 71 ___________________________________________
val_loss: 60521.35546875


Step: 72 ___________________________________________


2023-10-01 13:37:58.924393: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:37:58.957487: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:37:58.990566: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 27589.689453125


Step: 73 ___________________________________________
val_loss: 17062.357421875


Step: 74 ___________________________________________
val_loss: 53747.484375


Step: 75 ___________________________________________
val_loss: 19497.412109375


Step: 76 ___________________________________________
val_loss: 17052.234375


Step: 77 ___________________________________________
val_loss: 18124.796875


Step: 78 ___________________________________________
val_loss: 19982.89453125


Step: 79 ___________________________________________
val_loss: 17063.990234375


Step: 80 ___________________________________________
val_loss: 68411.34375


Step: 81 ___________________________________________
val_loss: 17684.763671875


Step: 82 ___________________________________________
val_loss: 17051.326171875


Step: 83 ___________________________________________
val_loss: 17815.201171875


Step: 84 ___________________________________________
val_loss: 17265.056640625


Step: 85 _____

2023-10-01 13:42:30.660938: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 59934.20703125


Step: 88 ___________________________________________
val_loss: 18256.7109375


Step: 89 ___________________________________________
val_loss: 17073.576171875


Step: 90 ___________________________________________
val_loss: 18804.169921875


Step: 91 ___________________________________________
val_loss: 19543.345703125


Step: 92 ___________________________________________
val_loss: 17647.328125


Step: 93 ___________________________________________
val_loss: 18541.220703125


Step: 94 ___________________________________________
val_loss: 67706.0703125


Step: 95 ___________________________________________
val_loss: 17052.296875


Step: 96 ___________________________________________
val_loss: 17053.17578125


Step: 97 ___________________________________________
val_loss: 17051.626953125


Step: 98 ___________________________________________
val_loss: 25321.583984375


Step: 99 ___________________________________________
val_loss: 18166.025390625


final_seed: 

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 457ms/step - loss: 1164276.7500 - val_loss: 699714.6875
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 1111318.2500 - val_loss: 28759.1406
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 1683857.6250 - val_loss: 852476.4375
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 1138163.5000 - val_loss: 386151.4062
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 1127533.0000 - val_loss: 764992.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 984877.3750 - val_loss: 289669.4062
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 1093918.5000 - val_loss: 1047307.4375
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 981397.5000 - val_loss: 405876.6562
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 635841.5625 - val_loss: 651348.8125
Epo

Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 132521.1719 - val_loss: 180448.8125
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 135482.1406 - val_loss: 183234.8750
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 132467.6250 - val_loss: 199376.0469
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 135411.5938 - val_loss: 171995.3438
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 130239.7656 - val_loss: 133400.7031
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 123959.4297 - val_loss: 125301.3125
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 159342.3906 - val_loss: 112688.5234
Epoch 82/10000
1/1 [==============================] - 0s 14ms/step - loss: 134356.2812 - val_loss: 120589.4609
Epoch 83/10000
1/1 [==============================] - 0s 14ms/step - loss: 152792.0312 - val_loss: 150418.4219
E

Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 57372.4570 - val_loss: 106386.2188
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 60280.1914 - val_loss: 73293.3984
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 65114.4648 - val_loss: 79528.6484
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 46120.5938 - val_loss: 71764.7578
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 51189.0391 - val_loss: 79115.4688
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 59082.0625 - val_loss: 75260.9062
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 58599.8320 - val_loss: 101724.7734
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 43703.2266 - val_loss: 75517.0625
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 54363.1562 - val_loss: 57240.1250
Epoch 15

1/1 [==============================] - 0s 14ms/step - loss: 50649.5234 - val_loss: 47112.5117
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 45541.3594 - val_loss: 46761.7461
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 47313.7539 - val_loss: 50911.1055
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 43736.6797 - val_loss: 49726.8242
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 51128.7344 - val_loss: 51104.2969
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 59617.4023 - val_loss: 38307.0195
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 58379.2188 - val_loss: 34429.4805
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 61636.0781 - val_loss: 37396.8633
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 53032.9453 - val_loss: 33947.8750
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 38348.5703 - val_loss: 49538.6055
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 37680.5508 - val_loss: 49476.5742
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 37706.2812 - val_loss: 49459.3750
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 35952.2031 - val_loss: 49637.7188
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 37164.3555 - val_loss: 49428.4062
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 35808.4688 - val_loss: 49523.4258
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 34140.9336 - val_loss: 49483.5430
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 35759.6484 - val_loss: 49512.8125
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 32260.8711 - val_loss: 50291.5430
Epoch 306/10000
1/1 [=====

...

1/1 [==============================] - 0s 13ms/step - loss: 14452.2900 - val_loss: 19730.3945
Epoch 1334/10000
1/1 [==============================] - 0s 13ms/step - loss: 17175.9160 - val_loss: 18880.9395
Epoch 1335/10000
1/1 [==============================] - 0s 14ms/step - loss: 12289.4141 - val_loss: 19273.4102
Epoch 1336/10000
1/1 [==============================] - 0s 14ms/step - loss: 13666.2734 - val_loss: 19067.7168
Epoch 1337/10000
1/1 [==============================] - 0s 14ms/step - loss: 12536.3955 - val_loss: 18532.2656
Epoch 1338/10000
1/1 [==============================] - 0s 13ms/step - loss: 13373.8633 - val_loss: 19194.0488
Epoch 1339/10000
1/1 [==============================] - 0s 14ms/step - loss: 12653.5234 - val_loss: 18942.8125
Epoch 1340/10000
1/1 [==============================] - 0s 13ms/step - loss: 11960.9795 - val_loss: 18463.0801
Epoch 1341/10000
1/1 [==============================] - 0s 13ms/step - loss: 13326.7588 - val_loss: 18731.2715
Epoch 1342/10000
1

Epoch 1407/10000
1/1 [==============================] - 0s 13ms/step - loss: 14241.2627 - val_loss: 17447.4766
Epoch 1408/10000
1/1 [==============================] - 0s 13ms/step - loss: 15128.4219 - val_loss: 17054.2188
Epoch 1409/10000
1/1 [==============================] - 0s 13ms/step - loss: 13726.6377 - val_loss: 17244.0156
Epoch 1410/10000
1/1 [==============================] - 0s 13ms/step - loss: 14744.8516 - val_loss: 17164.8262
Epoch 1411/10000
1/1 [==============================] - 0s 13ms/step - loss: 13189.3213 - val_loss: 17140.6094
Epoch 1412/10000
1/1 [==============================] - 0s 14ms/step - loss: 14016.1953 - val_loss: 17252.0254
Epoch 1413/10000
1/1 [==============================] - 0s 13ms/step - loss: 13539.3037 - val_loss: 17051.4082
Epoch 1414/10000
1/1 [==============================] - 0s 13ms/step - loss: 12934.5176 - val_loss: 17212.0996
Epoch 1415/10000
1/1 [==============================] - 0s 13ms/step - loss: 13092.3047 - val_loss: 17484.4395
E

Epoch 1481/10000
1/1 [==============================] - 0s 13ms/step - loss: 12414.3848 - val_loss: 17085.9824
Epoch 1482/10000
1/1 [==============================] - 0s 13ms/step - loss: 12879.0625 - val_loss: 17147.4551
Epoch 1483/10000
1/1 [==============================] - 0s 14ms/step - loss: 12097.0039 - val_loss: 17101.6523
Epoch 1484/10000
1/1 [==============================] - 0s 14ms/step - loss: 12021.5859 - val_loss: 17109.3828
Epoch 1485/10000
1/1 [==============================] - 0s 14ms/step - loss: 11514.4053 - val_loss: 17077.6387
Epoch 1486/10000
1/1 [==============================] - 0s 14ms/step - loss: 13983.7500 - val_loss: 17339.5234
Epoch 1487/10000
1/1 [==============================] - 0s 13ms/step - loss: 10974.2207 - val_loss: 17241.3926
Epoch 1488/10000
1/1 [==============================] - 0s 13ms/step - loss: 11193.8418 - val_loss: 17104.8105
Epoch 1489/10000
1/1 [==============================] - 0s 13ms/step - loss: 11093.2090 - val_loss: 17051.3262
E

Epoch 1555/10000
1/1 [==============================] - 0s 13ms/step - loss: 15607.8711 - val_loss: 29915.5312
Epoch 1556/10000
1/1 [==============================] - 0s 14ms/step - loss: 15533.4102 - val_loss: 29527.5332
Epoch 1557/10000
1/1 [==============================] - 0s 13ms/step - loss: 17023.3516 - val_loss: 30808.5156
Epoch 1558/10000
1/1 [==============================] - 0s 13ms/step - loss: 15346.7734 - val_loss: 29493.4160
Epoch 1559/10000
1/1 [==============================] - 0s 13ms/step - loss: 17180.9570 - val_loss: 29950.7051
Epoch 1560/10000
1/1 [==============================] - 0s 13ms/step - loss: 16184.4795 - val_loss: 29037.7051
Epoch 1561/10000
1/1 [==============================] - 0s 13ms/step - loss: 16273.2373 - val_loss: 28736.6172
Epoch 1562/10000
1/1 [==============================] - 0s 13ms/step - loss: 14903.5225 - val_loss: 29012.6660
Epoch 1563/10000
1/1 [==============================] - 0s 13ms/step - loss: 14775.2773 - val_loss: 28788.8906
E

Epoch 1629/10000
1/1 [==============================] - 0s 13ms/step - loss: 12915.2676 - val_loss: 17942.5332
Epoch 1630/10000
1/1 [==============================] - 0s 14ms/step - loss: 10891.7725 - val_loss: 17629.9980
Epoch 1631/10000
1/1 [==============================] - 0s 13ms/step - loss: 10640.3418 - val_loss: 17750.3926
Epoch 1632/10000
1/1 [==============================] - 0s 13ms/step - loss: 11672.2646 - val_loss: 17602.4434
Epoch 1633/10000
1/1 [==============================] - 0s 13ms/step - loss: 9585.8438 - val_loss: 17248.8105
Epoch 1634/10000
1/1 [==============================] - 0s 14ms/step - loss: 9247.2783 - val_loss: 17388.4375
Epoch 1635/10000
1/1 [==============================] - 0s 13ms/step - loss: 10828.9023 - val_loss: 17431.7305
Epoch 1636/10000
1/1 [==============================] - 0s 13ms/step - loss: 10629.3691 - val_loss: 17052.4102
Epoch 1637/10000
1/1 [==============================] - 0s 13ms/step - loss: 10185.5400 - val_loss: 18249.6973
Epo

In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 46ms/step
[1059.129](test_target) - [[902.70374]](prediction) = 156.4252646484374


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Maranhão - Desemprego,Maranhão - IDH,anhã-03o - Produção de Cimento (t),Maranhão - PIB - Estadual,Maranhão - PIB - Construção Civil,Maranhão - PIB - Per Capita,Maranhão - PIB - Preços de Mercado
0,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,1.841595,-1.715002,-1.158615,-1.559807,-1.739742,-1.772310,-1.736663
1,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,1.021603,-1.108574,-0.789722,-1.059405,-1.290087,-1.115454,-1.078298
2,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,0.742268,-0.561962,-0.790688,-0.587124,-0.288593,-0.475454,-0.481570
3,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.269150,-0.073882,-0.409716,-0.149665,1.030877,0.083272,0.035290
4,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.638280,0.356944,0.006330,0.216687,-0.165219,0.299453,0.298319
5,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.991426,0.731799,0.336310,0.579976,0.690588,0.800239,0.671216
6,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,-0.635514,1.051962,0.631477,0.821891,0.650720,0.719399,0.686159
7,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,-1.071096,1.318715,2.174624,1.737447,1.111456,1.460855,1.605547


train_target:


,Maranhão - Consumo de Cimento (t)
0,531.005
1,579.458
2,693.939
3,816.847
4,963.205
5,1123.586
6,1378.741
7,1282.016


test_input:


...

test_target:


,Maranhão - Consumo de Cimento (t)
17,1059.129


2023-10-01 13:49:37.917517: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:49:37.945849: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-01 13:49:37.972941: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


1/1 [==============================] - 0s 36ms/step
Error: 156.4252646484374




[1114.419677734375,
 1469.6962890625,
 1551.3502197265625,
 1336.561767578125,
 1159.275146484375,
 1198.633544921875,
 1388.7447509765625,
 1140.243408203125,
 841.023681640625,
 902.7037353515625]

In [21]:
display(targets)
display(predictions)

[1528.77,
 1563.125,
 1303.4115714285715,
 1191.5096571428571,
 1100.2177714285715,
 971.921,
 750.012,
 774.045,
 1038.249,
 1059.129]

[1114.419677734375,
 1469.6962890625,
 1551.3502197265625,
 1336.561767578125,
 1159.275146484375,
 1198.633544921875,
 1388.7447509765625,
 1140.243408203125,
 841.023681640625,
 902.7037353515625]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

254.51214541015625

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.2256235337698043